# VacationPy
----

#### Dependencies and Setup

In [ ]:
# dependencies and api key
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from api_keys import gkey

# urls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# data paths
city_data_path = "../output_data/cities.csv"

In [ ]:
# read in csv
city_data = pd.read_csv(city_data_path)

In [ ]:
# gmaps configuration
gmaps.configure(api_key=gkey)

### Humidity Heatmap

In [ ]:
# create function for heatmap
def heatmap(locations, weights):
    fig = gmaps.figure(center=(20,0), zoom_level=1.7)
    heat_layer = gmaps.heatmap_layer(locations, weights=weights)
    fig.add_layer(heat_layer)
    return fig

In [ ]:
# define locations and weights
locations = city_data[["Lat", "Lng"]]
weights = city_data["Humidity"]

# create and display the heatmap
humidity_heatmap = heatmap(locations, weights)
humidity_heatmap

#### Weather Data

In [ ]:
# create new data frame for weather specific cleanup
vacation_cities = city_data.dropna()

# find and drop all cities deemed not worthy
cold = vacation_cities.loc[vacation_cities["Max Temp"] < 65].index
vacation_cities = vacation_cities.drop(cold)
hot = vacation_cities.loc[vacation_cities["Max Temp"] > 85].index
vacation_cities = vacation_cities.drop(hot)
windy = vacation_cities.loc[vacation_cities["Wind Speed"] > 10].index
vacation_cities = vacation_cities.drop(windy)
cloudy = vacation_cities.loc[vacation_cities["Cloudiness"] > 0].index
vacation_cities = vacation_cities.drop(cloudy)
humid = vacation_cities.loc[vacation_cities["Humidity"] > 60].index
vacation_cities = vacation_cities.drop(humid)

# house this new data frame in yet another data frame
hotel_df = vacation_cities

In [ ]:
# list for all the responses
responses = []

# define default perameters
target_type = "lodging"
radius = 5000

# api request for each city
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    coords = f"{lat},{lng}"
    params = {
        "location": coords,
        "types": target_type,
        "radius": radius,
        "key": gkey
    }
    response = requests.get(base_url, params).json()
    responses.append(response)

In [ ]:
# create lists to house each response variable
hotel_names = []
hotel_addresses = []
hotel_ratings = []
hotel_lats = []
hotel_lngs = []

# loop through responses
for response in responses:
    hotel_names.append(response["results"][0]["name"])
    hotel_addresses.append(response["results"][0]["vicinity"])
    hotel_ratings.append(response["results"][0]["rating"])
    hotel_lats.append(response["results"][0]["geometry"]["location"]["lat"])
    hotel_lngs.append(response["results"][0]["geometry"]["location"]["lng"]) 

# create columns with new data
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Address"] = hotel_addresses
hotel_df["Hotel Rating"] = hotel_ratings
hotel_df["Hotel Lat"] = hotel_lats
hotel_df["Hotel Lng"] = hotel_lngs

### Hotel Map

In [ ]:
# create function for the final map
def hotel_map(fig, locations, info_box_content):
    marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_content)
    fig.add_layer(marker_layer)
    return fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Rating</dt><dd>{Hotel Rating} Stars</dd>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Hotel Address}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [ ]:
# define variables for function call
fig = humidity_heatmap
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

# create the final map
final_map = hotel_map(fig, hotel_locations, hotel_info)

# display the final map
final_map